## Imports

In [1]:
import pandas as pd
import numpy as np
import random

import xgboost as xgb

import matplotlib.pyplot as plt
from matplotlib import cm, colors
from matplotlib import patches as mpatches
from mpl_toolkits.basemap import Basemap

import seaborn as sns

import os, sys

import sklearn
from sklearn.utils import shuffle
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import r2_score, mean_squared_error

import pickle

from mpl_toolkits.basemap import Basemap

In [2]:
xgb.__version__

'0.90'

In [3]:
# Value to change to get correct paths
data_path = 'data/'

In [4]:
inputs_path = data_path + 'inputs/'
intermediate_path = data_path + 'intermediate/'
output_path = data_path + 'outputs/with_irrig/'

base_path = inputs_path + 'Base/'
diet_path = inputs_path + 'diet/'
pop_path = inputs_path + 'population/'
land_use_path = inputs_path + 'LU/'

data_2000_no_irrig_path = intermediate_path + 'Baseline_df_iterations/'
data_2000_path = intermediate_path + 'Baseline_df_iterations/with_irrig/'

#data_2050_no_irrig_path = intermediate_path + 'Future_dfs/All_change/'
data_2050_path = intermediate_path + 'Future_dfs/All_change+irrig/'

data_2050_gdp_cont_path = data_2050_path + 'gdp_cont/original/'
data_2050_gdp_income_path = data_2050_path + 'gdp_cat_income/original/'
data_2050_gdp_stat_path = data_2050_path + 'gdp_cat_stat/original/'

data_2050_gdp_cont_clustered_path = data_2050_path + 'gdp_cont/clustered/'
data_2050_gdp_income_clustered_path = data_2050_path + 'gdp_cat_income/clustered/'
data_2050_gdp_stat_clustered_path = data_2050_path + 'gdp_cat_stat/clustered/'

model_output_path = output_path + 'model_output/'
composite_path = output_path + 'composite/'
compare_path = output_path + 'compare/'
suff_path = output_path + 'sufficiency/'


figure_path = 'figures/with_irrig/'
dist_figures_path = figure_path + 'distributions/'
agg_assumption_fig_path = figure_path + 'agg_assumption/'
cluster_fig_path = figure_path + 'clusters/'
convex_hull_fig_path = figure_path + 'convex_hull/'
suff_fig_path = figure_path + 'sufficiency/'
model_fig_path = figure_path + 'models/'
sensitivity_fig_path = figure_path + 'sensitivity/'

model_path = data_path + 'models/'

In [5]:
numerical_col = ['slope', 'Annual Mean Temperature',
               'Mean Diurnal Range', 'Isothermality', 'Temperature Seasonality',
               'Max Temperature of Warmest Month', 'Min Temperature of Coldest Month',
               'Temperature Annual Range', 'Annual Precipitation',
               'Precipitation of Wettest Month', 'Precipitation of Driest Month',
               'Precipitation Seasonality', 'fertl_c3ann', 'fertl_c3nfx',
               'fertl_c3per', 'fertl_c4ann', 'log_altitude', 'irrig_c3ann', 
               'irrig_c3per', 'irrig_c3nfx', 'irrig_c4ann', 'irrig_c4per']

non_numerical_col = ['nutrient_availability_index', 'oxygen_availability_index',
               'workability_index', 'protected_areas_index', 'nutrient_retention_index', 
               'toxicity_index', 'excess_salts_index', 'rooting_conditions_index', 'cat_gdp']

In [6]:
files_2050 = os.listdir(data_2050_gdp_cont_path)
files_2050 = [filename for filename in files_2050 if 'ssp' in filename]

climate_models, climate_scenarios = [], []

for file in files_2050:
    if len(file) == 11:
        climate_models.append(file[:2])
        climate_scenarios.append(file[3:7])

climate_models = list(set(climate_models))
climate_models.sort()
climate_scenarios = list(set(climate_scenarios))
climate_scenarios.sort()
climate_models, climate_scenarios

(['cc', 'gs', 'he', 'mr'], ['ssp1', 'ssp2', 'ssp3', 'ssp4', 'ssp5'])

## Average value of yields

In [108]:
points_2000 = pd.read_csv(os.path.join(compare_path, 'ssp1'+'_compare.csv'), index_col='pixel_id').dropna(subset=['cal_per_ha_2000'])
points_2000.columns

Index(['%cropland_2000', '%cropland_2050', 'cal_per_ha_pred',
       'cal_per_ha_2000', 'delta_cal_per_ha', '%delta_cal_per_ha',
       'ha_per_pixel', 'population_2000', 'population_2050', 'calories_2000',
       'calories_2050', 'delta_cal', '%delta_cal', 'country_id', 'base_id',
       'ISO3', 'name', 'sovereignt'],
      dtype='object')

In [109]:
yields_2000 = points_2000['cal_per_ha_2000'].values
weights_2000 = (points_2000['%cropland_2000']*points_2000['ha_per_pixel']).values
np.average(yields_2000, weights=weights_2000)

7090645979.176232

In [110]:
points_2000['cal_per_ha_2000'].mean()

6947183570.025432

In [111]:
points_2000['cal_per_ha_2000'].sum()

6299497645791961.0

## Explore Namibia, Botswana and Sout Africa

In [32]:
countries = pd.read_csv(base_path + 'country_ids.csv')
countries_names = pd.read_csv(base_path + 'country_names.csv',encoding='latin-1')
countries = countries[['country_id','pixel_id']].merge(countries_names[['base_id','ISO3','name', 'sovereignt']],right_on='base_id',left_on='country_id',how='right')
countries = countries.set_index('pixel_id')

In [17]:
ha_per_cell = pd.read_csv(inputs_path + 'LU/ha_per_cell.csv', index_col='pixel_id')
land_use_2000 = pd.read_csv(inputs_path + 'LU/' + 'Cropmask_lulc_esa_2000.csv', index_col='pixel_id')['%cropland']

/Users/Romain/anaconda/envs/project/lib/python3.6/site-packages/numpy/lib/arraysetops.py:569: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [20]:
original_df = pd.read_csv(data_2000_path + 'baseline_df.csv', index_col='pixel_id', na_values=[-9999])
with_country = original_df.merge(countries, on='pixel_id')
with_country = with_country.merge(ha_per_cell, on='pixel_id')
with_country = with_country.merge(land_use_2000, on='pixel_id')
with_country['cropland_per_pixel'] = with_country['ha_per_pixel'] * with_country['%cropland']
with_country.head()

,log_calories_per_ha,nutrient_availability_index,oxygen_availability_index,workability_index,protected_areas_index,nutrient_retention_index,toxicity_index,excess_salts_index,rooting_conditions_index,slope,...,irrig_c4ann,irrig_c4per,country_id,base_id,ISO3,name,sovereignt,ha_per_pixel,%cropland,cropland_per_pixel
pixel_id,,,,,,,,,,,,,,,,,,,,,
1272549,22.911327,3.0,4.0,1.0,2,2.0,1.0,1.0,1.0,0.003900,...,0.0,0.0,73.0,73,FIN,Finland,Finland,3568.673828,0.052029,185.675017
1276867,22.911327,3.0,4.0,1.0,0,2.0,1.0,1.0,1.0,0.007459,...,0.0,0.0,73.0,73,FIN,Finland,Finland,3580.059326,0.016649,59.605566
1276868,22.911327,3.0,3.0,1.0,0,3.0,1.0,1.0,1.0,0.006798,...,0.0,0.0,73.0,73,FIN,Finland,Finland,3580.059326,0.071800,257.049005
1298466,22.911327,3.0,4.0,1.0,2,2.0,1.0,1.0,1.0,0.007925,...,0.0,0.0,73.0,73,FIN,Finland,Finland,3636.873047,0.225806,821.229398
1298467,22.911327,3.0,3.0,1.0,0,2.0,1.0,1.0,1.0,0.007998,...,0.0,0.0,73.0,73,FIN,Finland,Finland,3636.873047,0.097815,355.739923


In [ ]:
c_list = ['South Africa', 'Namibia', 'Botswana']

In [75]:
with_country['calories_per_ha'] = with_country['log_calories_per_ha'].apply(lambda x: np.exp(x))
with_country['calories'] = with_country['calories_per_ha'] * with_country['cropland_per_pixel']
avg_yield = with_country.groupby('name')['calories'].sum() / with_country.groupby('name')['cropland_per_pixel'].sum()
avg_yield[c_list]

name
South Africa    7.019445e+09
Namibia         2.966862e+09
Botswana        1.016966e+09
dtype: float64

In [77]:
for ssp in climate_scenarios:
    compare = pd.read_csv(compare_path + ssp + '_compare.csv', index_col='pixel_id')
    compare['cropland_per_pixel'] = compare['ha_per_pixel'] * compare['%cropland_2050']
    avg_yield = compare.groupby('name')['calories_2050'].sum() / compare.groupby('name')['cropland_per_pixel'].sum()
    print(ssp, avg_yield[c_list])

ssp1 name
South Africa    7.458437e+09
Namibia         5.673366e+09
Botswana        4.746781e+09
dtype: float64
ssp2 name
South Africa    7.615412e+09
Namibia         6.093808e+09
Botswana        4.841220e+09
dtype: float64
ssp3 name
South Africa    7.926830e+09
Namibia         5.809188e+09
Botswana        4.998198e+09
dtype: float64
ssp4 name
South Africa    7.409556e+09
Namibia         6.018751e+09
Botswana        4.672751e+09
dtype: float64
ssp5 name
South Africa    7.524208e+09
Namibia         5.679030e+09
Botswana        4.947836e+09
dtype: float64


In [61]:
c_list = ['South Africa', 'Namibia', 'Botswana']

sa_nam_bots = with_country[with_country['name'].isin(c_list)]
grouped_2000 = sa_nam_bots.groupby('name').mean()

In [64]:
res = pd.DataFrame(index=[climate_scenarios*3, c_list*5], columns=grouped_2000.columns)
res_2000 = pd.DataFrame(index=[['2000']*3, c_list])
res = res_2000.append(res, sort=False).sort_index(level=1)
res.loc[('2000', c_list), :] = grouped_2000.values
res

,,log_calories_per_ha,nutrient_availability_index,oxygen_availability_index,workability_index,protected_areas_index,nutrient_retention_index,toxicity_index,excess_salts_index,rooting_conditions_index,slope,...,irrig_c3ann,irrig_c3per,irrig_c3nfx,irrig_c4ann,irrig_c4per,country_id,base_id,ha_per_pixel,%cropland,cropland_per_pixel
2000,Botswana,20.8371,3.09414,1.13895,1.29147,0.5938,2.56891,1.1661,1.24236,1.27291,0.0116468,...,0.000293317,0.000293317,0.000293317,0.000293317,0.000293317,36,36,7966.51,0.0657354,524.114
ssp1,Botswana,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
ssp2,Botswana,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
ssp3,Botswana,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
ssp4,Botswana,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
ssp5,Botswana,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2000,Namibia,21.912,3.04663,1.61212,2.80662,0.115231,2.56017,1.61856,2.05417,2.79574,0.00772198,...,0.00538017,0.00538017,0.00538017,0.00538017,0.00538017,160,160,8034.35,0.0389572,318.474
ssp1,Namibia,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
ssp2,Namibia,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
ssp3,Namibia,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [69]:
for ssp in climate_scenarios:
    data_ssp = pd.DataFrame(index=c_list)
    for model in ['he']:
        df = pd.read_csv(data_2050_gdp_cont_path + '{}_{}.csv'.format(model, ssp), index_col='pixel_id', na_values=[-9999])
        
        with_country_2050 = df.merge(countries, on='pixel_id')
        with_country_2050 = with_country_2050.merge(ha_per_cell, on='pixel_id')
        with_country_2050['cropland_per_pixel'] = with_country_2050['ha_per_pixel'] * with_country_2050['%cropland']
        
        sa_nam_bots = with_country_2050[with_country_2050['name'].isin(c_list)]
        grouped = sa_nam_bots.groupby('name').mean()
        
        """for i, row in grouped.iterrows():
            data_ssp.loc[i] = row[grouped_2000.columns].values"""
    for country in c_list:    
        res.loc[(ssp, country)] = grouped.loc[country, grouped_2000.columns].values
            

        
res

,,log_calories_per_ha,nutrient_availability_index,oxygen_availability_index,workability_index,protected_areas_index,nutrient_retention_index,toxicity_index,excess_salts_index,rooting_conditions_index,slope,...,irrig_c3ann,irrig_c3per,irrig_c3nfx,irrig_c4ann,irrig_c4per,country_id,base_id,ha_per_pixel,%cropland,cropland_per_pixel
2000,Botswana,20.8371,3.09414,1.13895,1.29147,0.5938,2.56891,1.1661,1.24236,1.27291,0.0116468,...,0.000293317,0.000293317,0.000293317,0.000293317,0.000293317,36,36,7966.51,0.0657354,524.114
ssp1,Botswana,NaN,2.90954,1.19413,1.33891,0.669604,2.44905,1.20529,1.28076,1.32012,0.0125803,...,0.00355921,0.00355987,0.00352144,0.00384744,0.00340217,36,36,8006.96,0.126581,1014.53
ssp2,Botswana,NaN,3.06427,1.15757,1.27759,0.690624,2.56577,1.17738,1.24557,1.26353,0.0127766,...,0.00343436,0.00343436,0.00343436,0.00343436,0.00343436,36,36,7975.1,0.10164,813.449
ssp3,Botswana,NaN,3.20525,1.15033,1.27949,0.786931,2.66252,1.19972,1.26845,1.26476,0.0135764,...,0.0174126,0.0134231,0.01821,0.0183889,0.00748075,36,36,7978.94,0.411608,3287.91
ssp4,Botswana,NaN,2.96734,1.17964,1.31105,0.693034,2.4866,1.18831,1.25695,1.2914,0.0123893,...,0.00135948,0.00135948,0.00135948,0.00135948,0.00135948,36,36,7993.7,0.136377,1094.44
ssp5,Botswana,NaN,2.92259,1.20248,1.34435,0.694766,2.45537,1.22066,1.29394,1.32287,0.0127459,...,0.167288,0.057619,0.255935,0.305128,0.0122962,36,36,8010.2,0.145996,1172.26
2000,Namibia,21.912,3.04663,1.61212,2.80662,0.115231,2.56017,1.61856,2.05417,2.79574,0.00772198,...,0.00538017,0.00538017,0.00538017,0.00538017,0.00538017,160,160,8034.35,0.0389572,318.474
ssp1,Namibia,NaN,3.24673,1.60489,2.25412,0.15975,2.87635,1.61342,2.10347,2.24218,0.0105835,...,0.0458429,0.0369713,0.0403402,0.0479511,0.0334849,160,160,8094.93,0.0978985,799.827
ssp2,Namibia,NaN,3.19107,1.65628,2.67381,0.240659,2.74872,1.677,1.99575,2.67009,0.00767157,...,0.00813917,0.00813917,0.00813917,0.00813917,0.00813917,160,160,7983.54,0.0857933,686.837
ssp3,Namibia,NaN,3.17499,1.65163,2.55303,0.195594,2.76426,1.66761,2.08758,2.54415,0.00855748,...,0.0190136,0.0187674,0.0188171,0.0188477,0.0184232,160,160,8060.3,0.295584,2394.37


In [82]:
res.columns

Index(['log_calories_per_ha', 'nutrient_availability_index',
       'oxygen_availability_index', 'workability_index',
       'protected_areas_index', 'nutrient_retention_index', 'toxicity_index',
       'excess_salts_index', 'rooting_conditions_index', 'slope',
       'Annual Mean Temperature', 'Mean Diurnal Range', 'Isothermality',
       'Temperature Seasonality', 'Max Temperature of Warmest Month',
       'Min Temperature of Coldest Month', 'Temperature Annual Range',
       'Annual Precipitation', 'Precipitation of Wettest Month',
       'Precipitation of Driest Month', 'Precipitation Seasonality',
       'fertl_c3ann', 'fertl_c3nfx', 'fertl_c3per', 'fertl_c4ann',
       'fertl_c4per', 'log_altitude', 'log_GDP_pc', 'irrig_c3ann',
       'irrig_c3per', 'irrig_c3nfx', 'irrig_c4ann', 'irrig_c4per',
       'country_id', 'base_id', 'ha_per_pixel', '%cropland',
       'cropland_per_pixel'],
      dtype='object')

In [83]:
res.loc[:, res.columns.str.contains('irrig')]

,,irrig_c3ann,irrig_c3per,irrig_c3nfx,irrig_c4ann,irrig_c4per
2000,Botswana,0.000293317,0.000293317,0.000293317,0.000293317,0.000293317
ssp1,Botswana,0.00355921,0.00355987,0.00352144,0.00384744,0.00340217
ssp2,Botswana,0.00343436,0.00343436,0.00343436,0.00343436,0.00343436
ssp3,Botswana,0.0174126,0.0134231,0.01821,0.0183889,0.00748075
ssp4,Botswana,0.00135948,0.00135948,0.00135948,0.00135948,0.00135948
ssp5,Botswana,0.167288,0.057619,0.255935,0.305128,0.0122962
2000,Namibia,0.00538017,0.00538017,0.00538017,0.00538017,0.00538017
ssp1,Namibia,0.0458429,0.0369713,0.0403402,0.0479511,0.0334849
ssp2,Namibia,0.00813917,0.00813917,0.00813917,0.00813917,0.00813917
ssp3,Namibia,0.0190136,0.0187674,0.0188171,0.0188477,0.0184232


## Some irrigation


In [94]:
c_list = ['South Africa', 'Namibia', 'Botswana']

In [95]:
countries = pd.read_csv(base_path + 'country_ids.csv')
countries_names = pd.read_csv(base_path + 'country_names.csv',encoding='latin-1')
countries = countries[['country_id','pixel_id']].merge(countries_names[['base_id','ISO3','name', 'sovereignt']],right_on='base_id',left_on='country_id',how='right')
countries = countries.set_index('pixel_id')

In [100]:
# Mean ha irrigated in a cell

ha_per_cell = pd.read_csv(inputs_path + 'LU/ha_per_cell.csv', index_col='pixel_id')
land_use_2000 = pd.read_csv(inputs_path + 'LU/' + 'Cropmask_lulc_esa_2000.csv', index_col='pixel_id')['%cropland']
joined_df = original_df.join([ha_per_cell, land_use_2000, countries])

for country in c_list:
    joined_df_c = joined_df[joined_df['name'] == country]
    for col in joined_df.columns:
        if 'irrig' in col or 'fertl' in col:
            print('2000', country, col, (joined_df_c[col]*joined_df_c['ha_per_pixel']*joined_df_c['%cropland']).mean())
        if 'GDP' in col:
            print('2000', country, col, joined_df_c[col].values[0])
    print()

/Users/Romain/anaconda/envs/project/lib/python3.6/site-packages/numpy/lib/arraysetops.py:569: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


2000 South Africa fertl_c3ann 64839.3286536593
2000 South Africa fertl_c3nfx 24134.467027494535
2000 South Africa fertl_c3per 100675.39821544028
2000 South Africa fertl_c4ann 69136.55049999972
2000 South Africa fertl_c4per nan
2000 South Africa log_GDP_pc 2.2481320456241383
2000 South Africa irrig_c3ann 130.08096014457146
2000 South Africa irrig_c3per 130.08096014457146
2000 South Africa irrig_c3nfx 130.08096014457146
2000 South Africa irrig_c4ann 130.08096014457146
2000 South Africa irrig_c4per 130.08096014457146

2000 Namibia fertl_c3ann 3042.8101532372316
2000 Namibia fertl_c3nfx 368.4944845064227
2000 Namibia fertl_c3per 294.41228825103883
2000 Namibia fertl_c4ann 2266.1074371093296
2000 Namibia fertl_c4per nan
2000 Namibia log_GDP_pc 1.7699881949487009
2000 Namibia irrig_c3ann 2.3199828639173914
2000 Namibia irrig_c3per 2.3199828639173914
2000 Namibia irrig_c3nfx 2.3199828639173914
2000 Namibia irrig_c4ann 2.3199828639173914
2000 Namibia irrig_c4per 2.3199828639173914

2000 Botswa

In [101]:
cols = ['irrig_c3ann','irrig_c3per', 'irrig_c3nfx', 'irrig_c4ann', 'irrig_c4per', 
        'fertl_c3ann', 'fertl_c3nfx', 'fertl_c3per', 'fertl_c4ann',
        'fertl_c4per', 'log_GDP_pc'] 

for ssp in climate_scenarios:
    for country in c_list:
        for col in cols:
            data = []
            for model in ['he']:
                df = pd.read_csv(data_2050_gdp_cont_path + '{}_{}.csv'.format(model, ssp), index_col='pixel_id', na_values=[-9999])

                joined_df = df.join([ha_per_cell, countries])
                joined_df = joined_df[joined_df['name'] == country]
                data.append((joined_df[col]*joined_df['ha_per_pixel']*joined_df['%cropland']).mean())   
            if 'GDP' in col:
                print(ssp, country, col, joined_df[col].values[0])
            else:
                print(ssp, country, col, np.nanmean(data))
        print()

ssp1 South Africa irrig_c3ann 258.65650350524845
ssp1 South Africa irrig_c3per 258.817693556451
ssp1 South Africa irrig_c3nfx 258.6175674968931
ssp1 South Africa irrig_c4ann 258.6175674968931
ssp1 South Africa irrig_c4per 258.85640256010214
ssp1 South Africa fertl_c3ann 75797.0085584863
ssp1 South Africa fertl_c3nfx 28126.65483915413
ssp1 South Africa fertl_c3per 116827.46150087348
ssp1 South Africa fertl_c4ann 80222.4463490814


/Users/Romain/anaconda/envs/project/lib/python3.6/site-packages/ipykernel_launcher.py:18: RuntimeWarning: Mean of empty slice


ssp1 South Africa fertl_c4per nan
ssp1 South Africa log_GDP_pc 3.5048774151393034

ssp1 Namibia irrig_c3ann 34.274752104250425
ssp1 Namibia irrig_c3per 27.274935702161006
ssp1 Namibia irrig_c3nfx 30.40684537796566
ssp1 Namibia irrig_c4ann 38.24533873937661
ssp1 Namibia irrig_c4per 24.120900355174506
ssp1 Namibia fertl_c3ann 6697.252975913386
ssp1 Namibia fertl_c3nfx 864.7986204119723
ssp1 Namibia fertl_c3per 2234.2994685990852
ssp1 Namibia fertl_c4ann 5776.037909503163
ssp1 Namibia fertl_c4per nan
ssp1 Namibia log_GDP_pc 3.1367251368132765

ssp1 Botswana irrig_c3ann 2.60314762125537
ssp1 Botswana irrig_c3per 2.577646013705933
ssp1 Botswana irrig_c3nfx 2.536992590669222
ssp1 Botswana irrig_c4ann 3.122715801566505
ssp1 Botswana irrig_c4per 2.197561565816513
ssp1 Botswana fertl_c3ann 12060.62668237982
ssp1 Botswana fertl_c3nfx 3344.868463323541
ssp1 Botswana fertl_c3per 7772.632058702842
ssp1 Botswana fertl_c4ann 14855.95829829697
ssp1 Botswana fertl_c4per nan
ssp1 Botswana log_GDP_pc 3.1

In [92]:
original_df = pd.read_csv(data_2000_path + 'baseline_df.csv', index_col='pixel_id', na_values=[-9999])
ha_per_cell = pd.read_csv(inputs_path + 'LU/ha_per_cell.csv', index_col='pixel_id')
land_use_2000 = pd.read_csv(inputs_path + 'LU/' + 'Cropmask_lulc_esa_2000.csv', index_col='pixel_id')['%cropland']
joined_df = original_df.join([ha_per_cell, land_use_2000])
joined_df.head()

,log_calories_per_ha,nutrient_availability_index,oxygen_availability_index,workability_index,protected_areas_index,nutrient_retention_index,toxicity_index,excess_salts_index,rooting_conditions_index,slope,...,fertl_c4per,log_altitude,log_GDP_pc,irrig_c3ann,irrig_c3per,irrig_c3nfx,irrig_c4ann,irrig_c4per,ha_per_pixel,%cropland
pixel_id,,,,,,,,,,,,,,,,,,,,,
1272549,22.911327,3.0,4.0,1.0,2,2.0,1.0,1.0,1.0,0.003900,...,NaN,4.290459,3.456326,0.0,0.0,0.0,0.0,0.0,3568.673828,0.052029
1276867,22.911327,3.0,4.0,1.0,0,2.0,1.0,1.0,1.0,0.007459,...,NaN,3.891820,3.456326,0.0,0.0,0.0,0.0,0.0,3580.059326,0.016649
1276868,22.911327,3.0,3.0,1.0,0,3.0,1.0,1.0,1.0,0.006798,...,NaN,3.871201,3.456326,0.0,0.0,0.0,0.0,0.0,3580.059326,0.071800
1298466,22.911327,3.0,4.0,1.0,2,2.0,1.0,1.0,1.0,0.007925,...,NaN,2.944439,3.456326,0.0,0.0,0.0,0.0,0.0,3636.873047,0.225806
1298467,22.911327,3.0,3.0,1.0,0,2.0,1.0,1.0,1.0,0.007998,...,NaN,3.332205,3.456326,0.0,0.0,0.0,0.0,0.0,3636.873047,0.097815


In [93]:
joined_df[joined_df['nutrient_availability_index'] == 3]

,log_calories_per_ha,nutrient_availability_index,oxygen_availability_index,workability_index,protected_areas_index,nutrient_retention_index,toxicity_index,excess_salts_index,rooting_conditions_index,slope,...,fertl_c4per,log_altitude,log_GDP_pc,irrig_c3ann,irrig_c3per,irrig_c3nfx,irrig_c4ann,irrig_c4per,ha_per_pixel,%cropland
pixel_id,,,,,,,,,,,,,,,,,,,,,
1272549,22.911327,3.0,4.0,1.0,2,2.0,1.0,1.0,1.0,0.003900,...,NaN,4.290459,3.456326,0.0,0.0,0.0,0.0,0.0,3568.673828,0.052029
1276867,22.911327,3.0,4.0,1.0,0,2.0,1.0,1.0,1.0,0.007459,...,NaN,3.891820,3.456326,0.0,0.0,0.0,0.0,0.0,3580.059326,0.016649
1276868,22.911327,3.0,3.0,1.0,0,3.0,1.0,1.0,1.0,0.006798,...,NaN,3.871201,3.456326,0.0,0.0,0.0,0.0,0.0,3580.059326,0.071800
1298466,22.911327,3.0,4.0,1.0,2,2.0,1.0,1.0,1.0,0.007925,...,NaN,2.944439,3.456326,0.0,0.0,0.0,0.0,0.0,3636.873047,0.225806
1298467,22.911327,3.0,3.0,1.0,0,2.0,1.0,1.0,1.0,0.007998,...,NaN,3.332205,3.456326,0.0,0.0,0.0,0.0,0.0,3636.873047,0.097815
1298473,23.067026,3.0,4.0,1.0,0,3.0,1.0,1.0,1.0,0.018116,...,NaN,4.304065,3.456326,0.0,0.0,0.0,0.0,0.0,3636.873047,0.022893
1298474,23.067026,3.0,4.0,1.0,0,3.0,1.0,1.0,1.0,0.016824,...,NaN,4.317488,3.456326,0.0,0.0,0.0,0.0,0.0,3636.873047,0.030177
1302785,22.911327,3.0,5.0,1.0,0,2.0,1.0,1.0,1.0,0.025673,...,NaN,2.197225,3.456326,0.0,0.0,0.0,0.0,0.0,3648.212891,0.621228
1302786,22.911327,3.0,4.0,1.0,0,2.0,1.0,1.0,1.0,0.012379,...,NaN,2.564949,3.456326,0.0,0.0,0.0,0.0,0.0,3648.212891,0.714880


## Fertilizers numbers

In [24]:
original_df = pd.read_csv(data_2000_path + 'baseline_df.csv', index_col='pixel_id', na_values=[-9999])
ha_per_cell = pd.read_csv(inputs_path + 'LU/ha_per_cell.csv', index_col='pixel_id')
land_use_2000 = pd.read_csv(inputs_path + 'LU/' + 'Cropmask_lulc_esa_2000.csv', index_col='pixel_id')['%cropland']
joined_df = original_df.join([ha_per_cell, land_use_2000])

fert = joined_df[joined_df.str.contains('fert')]
print(fert.columns)

for model in climate_models:
    for ssp in climate_scenarios:
        df = pd.read_csv(data_2050_gdp_cont_path + '{}_{}.csv'.format(model, ssp), index_col='pixel_id', na_values=[-9999])
        land_use_2050 = df['%cropland']
        joined_df = original_df.join([ha_per_cell, df])

/Users/Romain/anaconda/envs/project/lib/python3.6/site-packages/numpy/lib/arraysetops.py:569: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


AttributeError: 'DataFrame' object has no attribute 'str'

In [25]:
ha_per_cell

,ha_per_pixel
pixel_id,
0,0.000015
1,0.000015
2,0.000015
3,0.000015
4,0.000015
5,0.000015
6,0.000015
7,0.000015
8,0.000015


## Cropland numbers

In [59]:
prod = df1['calories_2000'].sum()
prod_pred = df1['calories_2000_pred'].sum()

prod, prod_pred, (prod_pred - prod)/prod

(1.6849613311326798e+19, 1.6607802590008189e+19, -0.01435111399001998)

In [64]:
test = pd.read_csv(data_2050_gdp_stat_clustered_path + 'based2000_clustered_{}_{}_gdp_cat_new.csv'.format('he', 'ssp2'), index_col='pixel_id', na_values=[-9999]).drop(['log_GDP_pc', 'label_10'], axis=1)
print(len(test))
test_original = pd.read_csv(data_2050_gdp_cont_path + '{}_{}.csv'.format('he', 'ssp2'), index_col='pixel_id', na_values=[-9999])
print(len(test_original))

/Users/Romain/anaconda/envs/project/lib/python3.6/site-packages/numpy/lib/arraysetops.py:569: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


1278627
1278802


In [66]:
len(test) - len(test_original), (len(test) - len(test_original))/len(test_original)

(-175, -0.00013684683008002803)

In [8]:
sufficiencies = pd.read_csv(suff_path + 'country_sufficiencies.csv')
sufficiencies.head()

,ISO3,2000,ssp1,ssp2,ssp3,ssp4,ssp5
0,AFG,0.712556,0.536646,0.468967,0.390449,0.367319,0.655464
1,AGO,0.591377,0.453087,0.571087,0.381656,0.558107,0.686487
2,ALB,2.329763,2.544620,2.428040,2.353029,2.508106,2.448250
3,ARE,0.417912,0.513526,0.515186,0.525990,0.529507,0.520904
4,ARG,7.931488,5.890471,6.701443,6.249030,6.205168,6.367101


In [7]:
countries = pd.read_csv(base_path + 'country_ids.csv')
countries_names = pd.read_csv(base_path + 'country_names.csv',encoding='latin-1')
countries = countries[['country_id','pixel_id']].merge(countries_names[['base_id','ISO3','name', 'sovereignt']],right_on='base_id',left_on='country_id',how='right')

pop_dir = pop_path
pop = pd.read_csv(os.path.join(pop_dir,'pop_2000.csv'))
pop = pop.merge(countries, right_on='pixel_id', left_on='pixel_id')
pop_bycountry_2000 = pop.groupby('ISO3')['gpw_population'].sum()

prod = pd.read_csv(os.path.join(compare_path, 'ssp1'+'_compare.csv'), index_col='pixel_id', na_values=[-9999])
prod_bycountry_2000 = prod.groupby('ISO3')['calories_2000'].sum() 

/Users/Romain/anaconda/envs/project/lib/python3.6/site-packages/numpy/lib/arraysetops.py:569: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [14]:
def get_changes(country):
    changes = pd.DataFrame(columns=['2000', 'change_ssp1', 'change_ssp2', 'change_ssp3', 'change_ssp4', 'change_ssp5'], 
                           index=['pop', 'suff', 'prod', 'crop(%)'])
    
    
    changes.at['suff', '2000'] = sufficiencies[sufficiencies['ISO3'] == country]['2000'].values[0]
    changes.at['pop', '2000'] = pop_bycountry_2000[country]
    changes.at['prod', '2000'] = prod_bycountry_2000[country]
    
    df = pd.read_csv(os.path.join(compare_path, 'ssp1'+'_compare.csv'), index_col='pixel_id')
    df['cropland_2000'] = df.dropna(subset=['calories_2000'])['ha_per_pixel'] * df.dropna(subset=['calories_2000'])['%cropland_2000']
    changes.at['crop(%)', '2000'] = df.groupby('ISO3')['cropland_2000'].sum()[country]
    
    
    for ssp in climate_scenarios:
        name = 'change_' + ssp
        df = pd.read_csv(os.path.join(compare_path, ssp+'_compare.csv'), index_col='pixel_id', na_values=[-9999])
        
        changes.at['suff', name] = sufficiencies[sufficiencies['ISO3'] == country][ssp].values[0]
        
        pop = pd.read_csv(os.path.join(pop_dir,'pop_' + ssp + '.csv'))
        pop = pop.merge(countries, right_on='pixel_id', left_on='pixel_id')
        pop_bycountry = pop.groupby('ISO3')['gpw_population'].sum()
        changes.at['pop', name] = (pop_bycountry[country] - changes.at['pop', '2000']) / changes.at['pop', '2000']
        
        prod_bycountry = df.groupby('ISO3')['calories_2050'].sum() 
        changes.at['prod', name] = (prod_bycountry[country] - changes.at['prod', '2000']) / changes.at['prod', '2000']
        
        df['cropland_2050'] = df['ha_per_pixel'] * df['%cropland_2050']
        changes.at['crop(%)', name] = (df.groupby('ISO3')['cropland_2050'].sum()[country] - changes.at['crop(%)', '2000']) / changes.at['crop(%)', '2000']
    
    return changes 

In [84]:
# India
print('India')
get_changes('IND')

India


/Users/Romain/anaconda/envs/project/lib/python3.6/site-packages/numpy/lib/arraysetops.py:569: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


,2000,change_ssp1,change_ssp2,change_ssp3,change_ssp4,change_ssp5
pop,1.03828e+09,0.273267,0.175448,1.11563,0.299037,0.269964
suff,0.653553,0.588175,0.639735,0.421744,0.582959,0.639958
prod,1.1043e+18,0.301484,0.358178,0.454716,0.339789,0.470679
crop(%),2.18699e+08,-0.0517236,7.88868e-05,0.0626794,-0.0194453,0.0724131


In [56]:
# Brazil
print('Brazil')
get_changes('BRA')

Brazil


/Users/Romain/anaconda/envs/project/lib/python3.6/site-packages/numpy/lib/arraysetops.py:569: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


,2000,change_ssp1,change_ssp2,change_ssp3,change_ssp4,change_ssp5
pop,1.68511e+08,0.127831,0.127917,0.128668,0.128121,0.127831
suff,3.64748,3.31226,3.51964,3.36292,3.15167,4.12829
prod,1.00013e+18,0.24303,0.341857,0.304493,0.180426,0.606187
crop(%),2.00392e+08,0.0514303,0.133953,0.0994143,-0.00254422,0.356709


In [57]:
# France
print('France')
get_changes('FRA')

France


/Users/Romain/anaconda/envs/project/lib/python3.6/site-packages/numpy/lib/arraysetops.py:569: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


,2000,change_ssp1,change_ssp2,change_ssp3,change_ssp4,change_ssp5
pop,5.90026e+07,0.0683851,0.0686514,0.0771331,0.0709898,0.0734986
suff,5.40715,3.17037,3.25941,3.59111,3.15217,3.15161
prod,5.18418e+17,-0.359497,-0.339437,-0.268054,-0.360576,-0.40102
crop(%),2.84785e+07,-0.0879993,-0.0342982,0.0674864,-0.0690931,-0.0542632


In [58]:
# USA
print('USA')
get_changes('USA')

USA


/Users/Romain/anaconda/envs/project/lib/python3.6/site-packages/numpy/lib/arraysetops.py:569: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


,2000,change_ssp1,change_ssp2,change_ssp3,change_ssp4,change_ssp5
pop,2.75032e+08,0.0963864,0.0963863,0.0964063,0.0963863,0.968284
suff,5.52503,3.40373,3.5207,4.00427,4.0778,2.31963
prod,2.47476e+18,-0.216114,-0.189611,-0.0741521,-0.0184627,-0.197242
crop(%),1.89026e+08,-0.0871323,-0.0323887,0.0837837,0.162148,-0.0260027


In [59]:
# China
print('China')
get_changes('CHN')

China


/Users/Romain/anaconda/envs/project/lib/python3.6/site-packages/numpy/lib/arraysetops.py:569: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


,2000,change_ssp1,change_ssp2,change_ssp3,change_ssp4,change_ssp5
pop,1.25246e+09,0.0507877,0.0542715,0.0542755,0.0501957,0.0508146
suff,1.41497,1.29027,1.42406,1.25421,1.4072,1.45511
prod,2.88644e+18,-0.0808103,0.0620928,-0.0745808,0.0398189,0.0967482
crop(%),2.7464e+08,-0.158307,-0.0193483,-0.161715,-0.0355792,0.00453797


In [60]:
# Namibia
print('Namibia')
get_changes('NAM')

Namibia


/Users/Romain/anaconda/envs/project/lib/python3.6/site-packages/numpy/lib/arraysetops.py:569: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


,2000,change_ssp1,change_ssp2,change_ssp3,change_ssp4,change_ssp5
pop,1.81974e+06,0.499067,0.566891,1.01576,1.28288,0.494885
suff,1.93292,2.78807,3.81264,9.1961,2.91245,5.31636
prod,4.25569e+15,2.75114,4.55381,17.3979,5.05485,6.58276
crop(%),2.51646e+06,0.118156,0.541282,4.35591,0.701286,1.25805


In [63]:
# South Africa
print('South Africa')
get_changes('ZAF')

South Africa


/Users/Romain/anaconda/envs/project/lib/python3.6/site-packages/numpy/lib/arraysetops.py:569: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


,2000,change_ssp1,change_ssp2,change_ssp3,change_ssp4,change_ssp5
pop,4.42538e+07,0.14883,0.14888,0.149287,0.148935,0.66108
suff,1.5465,1.3234,1.34338,3.25445,1.27107,1.23845
prod,1.1005e+17,0.163359,0.188428,2.32602,0.110268,0.560456
crop(%),1.66556e+07,0.0306097,0.0311171,1.77239,-0.00993408,0.370311


In [61]:
# Bostswana
print('Bostswana')
get_changes('BWA')

Bostswana


/Users/Romain/anaconda/envs/project/lib/python3.6/site-packages/numpy/lib/arraysetops.py:569: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


,2000,change_ssp1,change_ssp2,change_ssp3,change_ssp4,change_ssp5
pop,1.70327e+06,0.170494,0.176752,0.1786,0.16583,0.189763
suff,1.38043,3.76267,3.88242,22.6173,4.5512,4.66115
prod,2.35535e+15,5.96184,6.25802,44.4839,7.50906,7.93901
crop(%),3.28096e+06,0.0528835,0.0762651,5.53281,0.307268,0.296969


In [115]:
# Mexico
get_changes('MEX')

/Users/Romain/anaconda/envs/project/lib/python3.6/site-packages/numpy/lib/arraysetops.py:569: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


,2000,change_ssp1,change_ssp2,change_ssp3,change_ssp4,change_ssp5
pop,1.01362e+08,0.152776,0.153486,1.14982,0.15392,0.155814
suff,1.43522,1.18683,1.26837,0.834653,1.37688,1.2154
prod,2.35589e+17,-0.0808405,0.0163119,-0.00895786,0.141756,-0.0437017
crop(%),3.23036e+07,-0.112762,-0.022751,-0.0393605,0.110326,-0.0791564


In [79]:
# Russia
get_changes('RUS')

/Users/Romain/anaconda/envs/project/lib/python3.6/site-packages/numpy/lib/arraysetops.py:569: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


,2000,change_ssp1,change_ssp2,change_ssp3,change_ssp4,change_ssp5
pop,1.45662e+08,-0.0347538,-0.0288861,-0.0299316,-0.0363294,-0.0325875
suff,3.81728,4.81829,4.90142,4.58175,4.83571,4.86473
prod,9.04621e+17,0.581047,0.623256,0.529835,0.584988,0.600544
crop(%),1.74761e+08,0.0413037,0.0643916,0.00526931,0.0380344,0.0534672


In [117]:
# Morocco
get_changes('MAR')

/Users/Romain/anaconda/envs/project/lib/python3.6/site-packages/numpy/lib/arraysetops.py:569: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


,2000,change_ssp1,change_ssp2,change_ssp3,change_ssp4,change_ssp5
pop,2.83191e+07,0.113823,0.113823,0.11468,0.113823,0.113823
suff,0.511462,1.08182,0.883328,0.828717,0.850426,0.624719
prod,2.21268e+16,2.15215,1.50013,1.37374,1.39939,0.630808
crop(%),9.06503e+06,0.275616,0.0412086,-0.00765226,0.00223082,-0.34363


In [118]:
# Algeria
get_changes('DZA')

/Users/Romain/anaconda/envs/project/lib/python3.6/site-packages/numpy/lib/arraysetops.py:569: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


,2000,change_ssp1,change_ssp2,change_ssp3,change_ssp4,change_ssp5
pop,3.04661e+07,0.155185,0.160759,1.05026,0.155294,0.155185
suff,0.440654,1.14768,0.996826,0.676069,0.938685,0.856801
prod,2.04009e+16,2.95242,2.38111,2.66655,2.12548,1.77977
crop(%),9.77611e+06,0.330851,0.152449,0.211195,0.100439,-0.0701567


In [15]:
# Netherlands
get_changes('NLD')

/Users/Romain/anaconda/envs/project/lib/python3.6/site-packages/numpy/lib/arraysetops.py:569: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


,2000,change_ssp1,change_ssp2,change_ssp3,change_ssp4,change_ssp5
pop,1.54964e+07,0.0477205,0.0477205,0.0476818,0.0477205,0.0477205
suff,0.678164,1.1744,1.1907,1.21616,1.19129,1.19658
prod,1.56492e+16,-0.15112,-0.11354,-0.0754566,-0.107484,-0.143293
crop(%),903441,-0.137198,-0.0958366,-0.0554623,-0.0880306,-0.109074


In [120]:
# Argentina
get_changes('ARG')

/Users/Romain/anaconda/envs/project/lib/python3.6/site-packages/numpy/lib/arraysetops.py:569: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


,2000,change_ssp1,change_ssp2,change_ssp3,change_ssp4,change_ssp5
pop,3.66853e+07,0.110495,0.111212,0.114619,0.113666,0.110282
suff,7.93149,5.89047,6.70144,6.24903,6.20517,6.3671
prod,4.7268e+17,-0.0044652,0.155744,0.0952234,0.0629888,0.0910136
crop(%),6.30798e+07,-0.0187832,0.144014,0.0831793,0.0522582,0.0430987


In [121]:
# Chile
get_changes('CHL')

/Users/Romain/anaconda/envs/project/lib/python3.6/site-packages/numpy/lib/arraysetops.py:569: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


,2000,change_ssp1,change_ssp2,change_ssp3,change_ssp4,change_ssp5
pop,1.48874e+07,0.118402,0.118854,0.119498,0.118421,0.118458
suff,2.21728,1.68913,1.79427,1.97391,1.9579,1.71404
prod,5.23152e+16,-0.0825614,-0.00661399,0.143,0.11322,-0.0715054
crop(%),5.00818e+06,-0.10664,-0.0168839,0.175596,0.0966375,-0.0814515


In [122]:
# Saudi Arabia
get_changes('SAU')

/Users/Romain/anaconda/envs/project/lib/python3.6/site-packages/numpy/lib/arraysetops.py:569: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


,2000,change_ssp1,change_ssp2,change_ssp3,change_ssp4,change_ssp5
pop,2.11984e+07,1.62517,1.63038,1.63169,1.63183,1.63939
suff,0.630262,0.54188,0.585503,0.689004,0.553992,0.544274
prod,2.02958e+16,0.486329,0.768705,1.46122,0.568797,0.48783
crop(%),2.58155e+06,0.010346,0.239145,0.711497,0.0982006,0.0317124


In [123]:
# Nigeria
get_changes('NGA')

/Users/Romain/anaconda/envs/project/lib/python3.6/site-packages/numpy/lib/arraysetops.py:569: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


,2000,change_ssp1,change_ssp2,change_ssp3,change_ssp4,change_ssp5
pop,1.21437e+08,1.58527,1.59766,2.6354,2.80373,1.58523
suff,0.955237,0.476895,0.533073,0.386782,0.351511,0.527983
prod,1.87557e+17,0.701304,0.956057,0.930791,0.800899,0.928792
crop(%),5.61477e+07,-0.00252155,0.153634,0.140215,0.0673081,0.147434


In [124]:
# Mozambique
get_changes('MOZ')

/Users/Romain/anaconda/envs/project/lib/python3.6/site-packages/numpy/lib/arraysetops.py:569: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


,2000,change_ssp1,change_ssp2,change_ssp3,change_ssp4,change_ssp5
pop,1.75943e+07,1.61295,1.66442,1.69749,1.68621,1.6029
suff,0.887487,0.587503,0.649156,0.407717,0.525732,0.691925
prod,2.39692e+16,1.34148,1.73236,0.623787,1.19763,1.85143
crop(%),1.10693e+07,-0.0459818,0.112081,0.0697549,0.452215,0.142454


In [125]:
# Angola
get_changes('AGO')

/Users/Romain/anaconda/envs/project/lib/python3.6/site-packages/numpy/lib/arraysetops.py:569: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


,2000,change_ssp1,change_ssp2,change_ssp3,change_ssp4,change_ssp5
pop,1.52012e+07,1.77479,1.78527,2.94943,3.10288,1.77517
suff,0.591377,0.453087,0.571087,0.381656,0.558107,0.686487
prod,1.31731e+16,1.71236,2.68496,2.14233,4.29108,3.53558
crop(%),7.62667e+06,-0.0567254,0.286455,0.0863966,0.844157,0.574578


In [126]:
# Niger
get_changes('NER')

/Users/Romain/anaconda/envs/project/lib/python3.6/site-packages/numpy/lib/arraysetops.py:569: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


,2000,change_ssp1,change_ssp2,change_ssp3,change_ssp4,change_ssp5
pop,1.12626e+07,2.93483,3.2408,4.58144,4.61001,1.89653
suff,0.673067,0.524753,0.427274,0.436517,0.396584,0.757167
prod,1.08769e+16,2.48528,1.89523,2.9988,2.50975,3.2411
crop(%),1.01508e+07,-0.170169,0.150994,0.357157,0.222587,0.00534217


In [127]:
# Madagascar
get_changes('MDG')

/Users/Romain/anaconda/envs/project/lib/python3.6/site-packages/numpy/lib/arraysetops.py:569: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


,2000,change_ssp1,change_ssp2,change_ssp3,change_ssp4,change_ssp5
pop,1.55127e+07,1.6522,1.66796,2.72722,2.70718,1.65198
suff,1.10678,0.629035,0.572517,0.312999,0.561928,0.614039
prod,2.65024e+16,0.886332,0.733926,0.0638388,1.39504,0.854447
crop(%),7.87904e+06,0.102856,0.021047,-0.0814454,1.00493,0.0788257


In [128]:
# Ghana
get_changes('GHA')

/Users/Romain/anaconda/envs/project/lib/python3.6/site-packages/numpy/lib/arraysetops.py:569: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


,2000,change_ssp1,change_ssp2,change_ssp3,change_ssp4,change_ssp5
pop,1.86764e+07,1.53347,1.57565,1.59155,1.57804,1.51137
suff,1.06665,0.581259,0.608303,0.523676,0.603281,0.776556
prod,3.09868e+16,0.836034,0.993141,0.70205,1.00477,1.55901
crop(%),1.12596e+07,-0.0058794,0.0774958,-0.0831293,0.0816111,0.367428


In [129]:
# DRC
get_changes('COD')

/Users/Romain/anaconda/envs/project/lib/python3.6/site-packages/numpy/lib/arraysetops.py:569: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


,2000,change_ssp1,change_ssp2,change_ssp3,change_ssp4,change_ssp5
pop,4.74098e+07,1.70056,1.72799,2.64925,2.68819,1.70054
suff,0.46579,0.577997,0.616675,0.278031,0.440682,0.537773
prod,3.45098e+16,3.09289,3.53785,1.29533,3.40783,2.7251
crop(%),2.7316e+07,0.0676602,0.178648,-0.0957541,0.499579,-0.0368966


In [130]:
# Kenya
get_changes('KEN')

/Users/Romain/anaconda/envs/project/lib/python3.6/site-packages/numpy/lib/arraysetops.py:569: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


,2000,change_ssp1,change_ssp2,change_ssp3,change_ssp4,change_ssp5
pop,3.04372e+07,1.57216,1.58849,2.22049,1.61504,1.56615
suff,1.04054,0.640422,0.764475,0.556241,0.673315,0.763321
prod,5.01344e+16,0.745013,1.2691,0.891152,0.999407,1.21615
crop(%),1.44202e+07,0.0563924,0.369611,0.135146,0.204049,0.321246


In [131]:
# Ethiopia
get_changes('ETH')

/Users/Romain/anaconda/envs/project/lib/python3.6/site-packages/numpy/lib/arraysetops.py:569: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


,2000,change_ssp1,change_ssp2,change_ssp3,change_ssp4,change_ssp5
pop,6.63836e+07,1.52664,1.5847,1.56993,1.61311,1.50518
suff,1.03491,0.525376,0.610268,0.513146,0.448758,0.834606
prod,1.10478e+17,0.6908,1.08907,0.74905,0.532318,1.83558
crop(%),2.80934e+07,0.0161277,0.294008,0.633286,0.432865,0.781471


In [132]:
# Tanzania
get_changes('TZA')

/Users/Romain/anaconda/envs/project/lib/python3.6/site-packages/numpy/lib/arraysetops.py:569: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


,2000,change_ssp1,change_ssp2,change_ssp3,change_ssp4,change_ssp5
pop,3.33032e+07,1.6374,1.66135,2.69788,2.77858,1.63
suff,1.36481,0.802822,0.879453,0.567635,0.62196,1.76561
prod,7.25922e+16,0.905735,1.19561,0.962506,1.24844,3.71857
crop(%),2.75364e+07,-0.103429,0.046819,-0.0672107,0.0669565,1.12676


In [88]:
get_changes('BEL')

/Users/Romain/anaconda/envs/project/lib/python3.6/site-packages/numpy/lib/arraysetops.py:569: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


,2000,change_ssp1,change_ssp2,change_ssp3,change_ssp4,change_ssp5
pop,1.04581e+07,0.0610784,0.0610784,0.0610784,0.0610784,0.0675167
suff,1.34768,1.49778,1.57623,1.65682,1.52548,1.57046
prod,2.14914e+16,-0.124346,-0.0276935,-0.00872715,-0.0835634,-0.1173
crop(%),1.67128e+06,-0.150744,-0.0585323,-0.0275923,-0.10075,-0.101177


In [ ]:
diets = pd.read_csv(diet_path + 'pctLS_SSPs_Bodirsky.csv',sep=';')

## Points in 2000 not in 2050

In [87]:
for ssp in climate_scenarios:
    df = pd.read_csv(os.path.join(compare_path, ssp+'_compare.csv'), index_col='pixel_id')
    points = df['cal_per_ha_2000'].dropna().index
    points_2050 = df['cal_per_ha_pred'].dropna().index

    in_2000_not_in_2050 = [i for i in points if i not in points_2050]
    print(ssp, len(points), len(points_2050), len(in_2000_not_in_2050), len(in_2000_not_in_2050)/len(points))

/Users/Romain/anaconda/envs/project/lib/python3.6/site-packages/numpy/lib/arraysetops.py:569: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


ssp1 906770 1060827 111029 0.12244450081056939
ssp2 906770 1278627 83010 0.09154471365395855
ssp3 906770 1072690 107614 0.11867838591925185
ssp4 906770 1190324 73562 0.0811253129238947
ssp5 906770 1066693 103687 0.11434762949810867


In [99]:
files_lu_2050 = ['Cropmask_lulc_RCP45_ssp2.csv',
                 'Cropmask_lulc_RCP60_ssp3.csv',
                 'Cropmask_lulc_RCP85_ssp5.csv',
                 'Cropmask_lulc_RCP60_ssp4.csv',
                 'Cropmask_lulc_RCP26_ssp1.csv']

In [113]:
points_2000 = pd.read_csv(os.path.join(compare_path, 'ssp1'+'_compare.csv'), index_col='pixel_id')
points_2000 = points_2000.dropna(subset=['calories_2000']).index

lu_2000 = pd.read_csv(land_use_path + 'Cropmask_lulc_esa_2000.csv', index_col='pixel_id')['%cropland']

for file in files_lu_2050:
    ssp = file[-8:-4]
    lu_ssp = pd.read_csv(land_use_path + file, index_col='pixel_id')['%cropland']
    lu_2050_index = lu_ssp[lu_ssp > 0].index
    lu_2000_index = lu_2000[lu_2000 > 0].index
    not_in_2050 = [i for i in points_2000 if i not in lu_2050_index]
    not_in_lu_2050 = [i for i in lu_2000_index if i not in lu_2050_index]
    
    print(ssp, len(not_in_2050), len(not_in_lu_2050))

/Users/Romain/anaconda/envs/project/lib/python3.6/site-packages/numpy/lib/arraysetops.py:569: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


ssp2 83010 21179
ssp3 107614 27373
ssp5 103687 18364
ssp4 73562 17063
ssp1 111029 24087


## Diet exploration

In [114]:
diets = pd.read_csv(diet_path + 'pctLS_SSPs_Bodirsky.csv',sep=';')

In [124]:
diet_2000 = pd.read_csv(diet_path + 'Consumption_2000_FAOSTAT.csv')
# LS 2000 Data
LS_2000_df = diet_2000[diet_2000['Item']=='Animal Products'][['Country','Value']].merge(countries[['name','ISO3']].drop_duplicates(),
                                                                                        right_on='name',left_on='Country',how='left')
# kcal/capita/day to cal/capita
LS_2000_df['LS2000_percapita'] = LS_2000_df['Value'].apply(lambda x:x*1e3*365.25)


# Population by country
pop_dir = pop_path
pop = pd.read_csv(os.path.join(pop_dir,'pop_2000.csv'))
pop = pop.merge(countries, right_on='pixel_id', left_on='pixel_id')
pop_bycountry_2000 = pop.groupby('ISO3')['gpw_population'].sum()
total_pop_2000 = pop['gpw_population'].sum()



LS_2000_df = LS_2000_df[['name','ISO3','LS2000_percapita']].set_index('ISO3')

In [132]:
LS_2000_df.at['NLD', 'LS2000_percapita']

431725500.0

In [133]:
diets[(diets['Scenario'] == 'ssp1')]['2000'].mean()

0.1603921052631579

In [117]:
diets[(diets['Scenario'] == 'ssp1')]['2050'].mean()

0.22632789473684214

In [139]:
diets[(diets['Scenario'] == 'ssp1') & (diets['Region'] == 'NLD')]['2000'].values[0]

0.3395

In [137]:
for ssp in climate_scenarios:
    print(diets[(diets['Scenario'] == ssp) & (diets['Region'] == 'NLD')]['2050'].values[0])

0.2361
0.2375
0.2568
0.2337
0.2756
